# TensorFlow 2 API로 모델 구성하기 / CIFAR-100 적용편
---
#### 3가지 방법을 모두 적용
* Sequential
* Functional : Sequential의 일반화된 개념
* Model Subclassing : 클래스로 구현된 기존의 모델을 상속받아 자신만의 모델을 생성
---

## 1. Sequential API
</br>


### 1-1. 데이터 호출

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

print(x_train.shape)

169001437/169001437 [==============================] - 138s 1us/step
50000 10000
(50000, 32, 32, 3)


### 1-2. Model 구성

>Spec:
>>1. 16개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
>>2. pool_size가 2인 MaxPool 레이어
>>3. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
>>4. pool_size가 2인 MaxPool 레이어
>>5. Flatten 레이어
>>6. 256개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
>>7. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)


In [3]:
model = keras.Sequential([
    keras.layers.Conv2D(16, 3, activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(100, activation='softmax')
])

Metal device set to: Apple M2


2022-09-18 00:54:35.218700: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-18 00:54:35.218915: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


### 1-3. 모델 학습 설정

In [4]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5


2022-09-18 00:54:41.030265: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-09-18 00:54:41.288478: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1563/1563 [==============================] - 13s 8ms/step - loss: 3.6377 - accuracy: 0.1512
Epoch 2/5
1563/1563 [==============================] - 11s 7ms/step - loss: 2.9827 - accuracy: 0.2678
Epoch 3/5
1563/1563 [==============================] - 11s 7ms/step - loss: 2.6986 - accuracy: 0.3224
Epoch 4/5
1563/1563 [==============================] - 12s 7ms/step - loss: 2.4979 - accuracy: 0.3645
Epoch 5/5
1563/1563 [==============================] - 12s 7ms/step - loss: 2.3306 - accuracy: 0.3993


2022-09-18 00:55:40.295953: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


313/313 - 1s - loss: 2.6496 - accuracy: 0.3387 - 1s/epoch - 3ms/step


[2.6495606899261475, 0.3387000262737274]

#### + model.compile의 속성을 따로 정의하고 넣어준 같은 코드

```Python
# 1.loss function(손실함수) 만들기
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
# 2.optimizer 만들기(여기선 Adam 사용)
optimizer = tf.keras.optimizers.Adam()

model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, verbose=2)
```
* from_logits=False : 
<br>* Softmax 함수를 사용하여 출력값으로 **해당 클래스의 범위에서의 확률**을 출력했으므로 from_logits=False (대체로 class 분류 문제)
<br>* 반대로 모델의 출력값이 sigmoid 또는 linear를 거쳐서 **확률이 아닌 값**이 나오게 된다면, from_logits=True
<br><br>


* 참고
>[from_logits란?](https://hwiyong.tistory.com/335)

In [5]:
%reset

# 메모리를 위한 코드

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


---

## 2. Functional API
</br>


### 2-1. 데이터 호출(1-1과 동일)

In [6]:
import tensorflow as tf
from tensorflow import keras

In [7]:
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

print(x_train.shape)

50000 10000
(50000, 32, 32, 3)


### 2-2. Model 구성

>Spec:
>>1. Input Laper: shape=
>>2. Flatten 레이어
>>3. 256개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
>>4. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)


In [8]:
inputs = keras.Input(shape=(32, 32, 3))

x = keras.layers.Conv2D(16, 3, activation='relu')(inputs)
x = keras.layers.MaxPool2D((2,2))(x)
x = keras.layers.Conv2D(32, 3, activation='relu')(x)
x = keras.layers.MaxPool2D((2,2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256, activation='relu')(x)
predictions = keras.layers.Dense(100, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=predictions)

### 2-3. 모델 학습 설정(1-3과 동일)

In [9]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
  15/1563 [..............................] - ETA: 12s - loss: 4.6179 - accuracy: 0.0104

2022-09-18 01:05:30.119713: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1563/1563 [==============================] - 12s 8ms/step - loss: 3.6229 - accuracy: 0.1583
Epoch 2/5
1563/1563 [==============================] - 12s 7ms/step - loss: 2.8948 - accuracy: 0.2874
Epoch 3/5
1563/1563 [==============================] - 12s 7ms/step - loss: 2.5889 - accuracy: 0.3466
Epoch 4/5
1563/1563 [==============================] - 12s 8ms/step - loss: 2.3876 - accuracy: 0.3885
Epoch 5/5
1563/1563 [==============================] - 12s 7ms/step - loss: 2.2181 - accuracy: 0.4235


2022-09-18 01:06:29.874322: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


313/313 - 1s - loss: 2.5724 - accuracy: 0.3534 - 1s/epoch - 3ms/step


[2.5723557472229004, 0.35340002179145813]

In [10]:
%reset

# 메모리 관리

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


---
## 3. Subclassing 활용
</br>


### 3-2. Model 구성만 진행(3-1과 3-3은 위와 동일)

>Spec:
>>0. keras.Model 을 상속받았으며, __init__()와 call() 메서드를 가진 모델 클래스
>>1. self.flatten = Flatten 레이어
>>2. self.linear = 256개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
>>3. self.linear2 = 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
>>4. call의 입력값이 모델의 Input, call의 리턴값이 모델의 Output


```Python
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16, 3, activation='relu')
        self.maxpool1 = keras.layers.MaxPool2D((2,2))
        self.conv2 = keras.layers.Conv2D(32, 3, activation='relu')
        self.maxpool2 = keras.layers.MaxPool2D((2,2))
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(256, activation='relu')
        self.fc2 = keras.layers.Dense(100, activation='softmax')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x
        
model = CustomModel()
```

---

# 심화 : GradientTape 활용

지금까지 3가지 방식으로 모델을 작성했지만 모델 학습 부분은 동일하게 진행하였음
```Python
# 모델 학습 설정
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
```

### 그렇다면 model.fit() 내부에서 실제로 훈련이 되는 과정은?

> 1. Forward Propagation 수행 및 중간 레이어값 저장
> 2. Loss 값 계산
> 3. 중간 레이어값 및 Loss를 활용한 체인룰(chain rule) 방식의 역전파(Backward Propagation) 수행
> 4. 학습 파라미터 업데이트

이상의 4단계로 이루어진 train_step을 여러 번 반복하게 됨

### * 여기서 GradientTape의 개념이 등장
> 순전파(forward pass)로 진행된 모든 연산의 중간 레이어값을 **tape**에 기록하고, 이를 이용해 gradient를 계산한 뒤 **tape**를 폐기
 
backpropagation을 할 때 forward propagation에서 gradient 값들을 저장해두면 이를 이용해 훨씬 빠르게 연산을 할 수 있음. 따라서 forward propagation이 진행되는 동안 그 값들을 저장할 필요가 있음. 즉, prediction을 구하는 과정과 loss를 구하는 과정이 tf.GradientTape의 대상이 되는 것.

In [11]:
# 앞서 구성한 Subclassing 모델

import tensorflow as tf
import numpy as np
from tensorflow import keras

# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

# 모델 구성부분
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16, 3, activation='relu')
        self.maxpool1 = keras.layers.MaxPool2D((2,2))
        self.conv2 = keras.layers.Conv2D(32, 3, activation='relu')
        self.maxpool2 = keras.layers.MaxPool2D((2,2))
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(256, activation='relu')
        self.fc2 = keras.layers.Dense(100, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

model = CustomModel()

50000 10000


### GradientTape를 적용한 모델 학습

tape.gradient()를 통해 매 스텝 학습이 진행될 때마다 발생하는 그래디언트를 추출한 후 optimizer.apply_gradients()를 통해 발생한 그래디언트가 업데이트해야 할 파라미터 model.trainable_variables를 지정해 주는 과정.

비교 : (적용하지 않았을 때)
```Python
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
```

#### * step마다 gradient를 추출하는 함수를 작성

In [12]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# tf.GradientTape()를 활용한 train_step
def train_step(features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features) #미니배치만큼 데이터를 가져온 뒤 경사를 초기화한 후 순방향전파
        loss = loss_func(labels, predictions) #손실을 계산
        gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

이렇게 매 스텝 진행되는 학습의 실제 동작이 train_step() 메서드로 구현됨
<br>
<br>

#### * tape를 적용하고 model.fit()를 대체하는 함수 작성
model.fit()으로 위와 같이 한 줄로 간단하게 수행되던 실제 배치 학습 과정은, 매 스텝마다 위에서 구현했던 train_step()가 호출되는 과정으로 바꾸어 구현할 수 있음

In [13]:
import time
def train_model(batch_size=32):
    start = time.time()
    for epoch in range(5):
        x_batch = []
        y_batch = []
        for step, (x, y) in enumerate(zip(x_train, y_train)):
            x_batch.append(x)
            y_batch.append(y)
            if step % batch_size == batch_size-1:
                loss = train_step(np.array(x_batch, dtype=np.float32), np.array(y_batch, dtype=np.float32))
                x_batch = []
                y_batch = []
        print('Epoch %d: last batch loss = %.4f' % (epoch, float(loss)))
    print("It took {} seconds".format(time.time() - start))

train_model()

Epoch 0: last batch loss = 3.2754
Epoch 1: last batch loss = 2.6648
Epoch 2: last batch loss = 2.3342
Epoch 3: last batch loss = 2.0787
Epoch 4: last batch loss = 1.9425
It took 77.94227981567383 seconds


#### => 위에서 구현한 train_model() 메서드가 그동안 사용했던 model.fit() 메서드와 기능적으로 같은 것을 확인할 수 있다.
**이렇듯 tf.GradientTape()를 활용하면 model.compile()과 model.fit() 안에 감추어져 있던 한 스텝의 학습 단계(위 예제에서는 train_step 메서드)를 끄집어내서 자유롭게 재구성할 수 있음.**<br>
<br>


여러 다른 강화학습 또는 GAN(Generative Advasarial Network)의 학습을 위해서는 train_step 메서드의 재구성이 필수적이므로 tf.GradientTape()의 활용법을 꼭 숙지해야 한다.

In [14]:
# 평가 단계
# gradient가 필요 없으므로 기존의 model.predict() 메서드를 이용

prediction = model.predict(x_test, batch_size=x_test.shape[0], verbose=1)
temp = sum(np.squeeze(y_test) == np.argmax(prediction, axis=1))
temp/len(y_test)  # Accuracy

2022-09-18 01:30:59.121117: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 315ms/step


0.3413